## Orchestration

Une application utilisant des conteneurs demande d'en coordonner plusieurs c'est que l'on appelle l'orchestration.
L'outil classique est [kubernetes](https://kubernetes.io/) de Google. Il s'agit d'un outils complexe. Une alternative simple est [docker compose](https://docs.docker.com/compose/).  

Docker compose permet de manipuler une architecture logicielle basée sur des conteneurs en décrivant des `services` dans un fichier YAML (`docker-compose.yml`). Les volumes et réseaux sont aussi gérés.
Docker Compose reprend les options des commandes de docker client. Docker compose est un simple plugin de docker client à [installer](https://docs.docker.com/compose/install/).

### Un exemple simple

Dans l'exemple suivant une application est composée de deux services (1) `db` une base de données relationnelles et (2) `app` une application JPA/REST écrit en Java.

Le service `app` s'appuie sur l'exécution d'une image docker de PostgreSQL en lui fixant des variables d'environnements, des volumes et une configuration réseau. 
Il faut noter que les volumes et les réseaux doivent externes définis globalement (dans le cas d'une utilisation avancée il est possible de les contrôler finement). 

```{literalinclude} sample-java/restjpa/Docker-compose.yml
:language: docker-compose
:caption: sample-java/restjpa/Docker-compose.yml
:name: sample-java/restjpa/Docker-compose.yml
```

In [71]:
( cd sample-java/restjpa && docker compose down --volumes --remove-orphans )

[+] Running 0/0
 ⠋ Container restjpa-app-1  Stopping                                       0.0s
[+] Running 0/1
 ⠙ Container restjpa-app-1  Stopping                                       0.1s
[+] Running 0/1
 ⠹ Container restjpa-app-1  Stopping                                       0.2s
[+] Running 0/1
 ⠸ Container restjpa-app-1  Stopping                                       0.3s
[+] Running 0/1
 ⠼ Container restjpa-app-1  Stopping                                       0.4s
[+] Running 0/1
 ⠴ Container restjpa-app-1  Stopping                                       0.5s
[+] Running 1/1
 ⠿ Container restjpa-app-1  Removed                                        0.6s
 ⠋ Container restjpa-db-1   Stopping                                       0.0s
[+] Running 2/2
 ⠿ Container restjpa-app-1         Removed                                 0.6s
 ⠿ Container restjpa-db-1          Removed                                 0.1s
 ⠋ Volume restjpa_restjpa-pg-data  Rem...                               

Il est possible de lancer explicitement un service avec la commande `docker compose up` depuis le répertoire qui contient le  l'option `-d` permet de le détacher.

In [74]:
( cd sample-java/restjpa && docker compose up --detach db )

la commande `ls` liste les projets en cours d'exécution.

In [76]:
docker compose ls -a

NAME                STATUS              CONFIG FILES
restjpa             running(1)          /home/jovyan/work/notebook-docker/sample-java/restjpa/docker-compose.yml


la commande `ps` liste les conteneurs du projet courant, noter que compose nomme automatiquement le conteneur (`<nom_repertoire_parent>-<service>-<id>`). 
Il est donc simple de lancer plusieurs fois le même projet depuis des répertoires différents ou plusieurs fois le même service sans risque de conflit à condition de ne pas utiliser de bind volumes ni de mapper des ports de l'hôte. 

In [78]:
( cd sample-java/restjpa && docker compose ps -a)

NAME                IMAGE                  COMMAND                  SERVICE             CREATED             STATUS              PORTS
restjpa-db-1        postgres:15.2-alpine   "docker-entrypoint.s…"   db                  15 seconds ago      Up 15 seconds       5432/tcp


On remarque que docker compose prend en charge la création et le nommage des réseaux et des volumes en les préfixant du nom du répertoire parent (le projet).

In [79]:
docker network ls

NETWORK ID     NAME               DRIVER    SCOPE
25a6bacef95b   bridge             bridge    local
290659c62698   host               host      local
e91b5de18bd5   none               null      local
e5221d449c6d   restjpa_backend    bridge    local
982774aa1614   restjpa_frontend   bridge    local


In [80]:
docker volume ls

DRIVER    VOLUME NAME
local     restjpa_restjpa-pg-data


L'affichage de log d'un service ou de tous les services se fait la commande `logs` (ici limité à 5 lignes).

In [83]:
( cd sample-java/restjpa && docker compose logs --tail 5)

restjpa-db-1  | 2023-03-10 16:52:31.337 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
restjpa-db-1  | 2023-03-10 16:52:31.340 UTC [52] LOG:  database system was shut down at 2023-03-10 16:52:31 UTC
restjpa-db-1  | 2023-03-10 16:52:31.343 UTC [1] LOG:  database system is ready to accept connections
restjpa-db-1  | 2023-03-10 16:57:31.373 UTC [50] LOG:  checkpoint starting: time
restjpa-db-1  | 2023-03-10 16:57:35.601 UTC [50] LOG:  checkpoint complete: wrote 44 buffers (0.3%); 0 WAL file(s) added, 0 removed, 0 recycled; write=4.211 s, sync=0.007 s, total=4.229 s; sync files=12, longest=0.005 s, average=0.001 s; distance=252 kB, estimate=252 kB


On peut stopper, détruite ou relancer un service avec `stop`, `rm` et `restart`. 
La commande `down` détruit tous les containeurs et les réseaux. Attention, l'option `-v` supprime les volumes associés nommés ou non. 

In [ ]:
( cd sample-java/restjpa && docker compose down -v )

[+] Running 0/0
 ⠋ Container restjpa-db-1  Stopping                                        0.0s
[+] Running 0/1
 ⠙ Container restjpa-db-1  Stopping                                        0.1s
[+] Running 1/1
 ⠿ Container restjpa-db-1          Removed                                 0.2s
 ⠋ Volume restjpa_restjpa-pg-data  Rem...                                  0.1s
 ⠋ Network restjpa_backend         Removing                                0.1s
 ⠋ Network restjpa_frontend        Removing                                0.1s
[+] Running 4/4
 ⠿ Container restjpa-db-1          Removed                                 0.2s
 ⠿ Volume restjpa_restjpa-pg-data  Rem...                                  0.1s
 ⠿ Network restjpa_backend         Removed                                 0.1s
 ⠿ Network restjpa_frontend        Removed                                 0.1s


L'autre service `app` est une application JPA/REST Java dont l'image docker est produite par `sample-java/restjpa/Dockerfile`.
L'option `build` dans `docker-compose.yml` indique qu'il faut fabriquer l'image à partir du contexte courant (`image` sera alors son tag).
La fabrication de l'image sera automatique au démarrage au besoin elle peut être faite manuellement avec `docker compose build`. 

In [ ]:
( cd sample-java/restjpa && \
  docker compose build --progress=quiet && \
  docker compose up --quiet-pull --detach )

Il est possible de contrôler l'ordre d'exécution des conteneurs avec la directive `depends_on` dans `docker-compose.yml` pour indiquer quels services doivent être démarrés avant un service. Dans l'exemple, la base de données doit être démarrée avant l'application.

In [4]:
docker run --network restjpa_frontend \
           --rm curlimages/curl:7.88.1 \
               curl --verbose --silent \
                 -H "Accept: application/json" \
                 http://restjpa-app-1:8080/restjpa/persons

Unable to find image 'curlimages/curl:7.88.1' locally
7.88.1: Pulling from curlimages/curl

e9b68314: Pulling fs layer 
07850c51: Pulling fs layer 
0a9d908e: Pulling fs layer 
6a508d84: Pulling fs layer 
ee6e30cf: Pulling fs layer 
69f78515: Pulling fs layer 
9ae18b39: Pulling fs layer 
4804c88f: Pulling fs layer 
6a1158e3: Pulling fs layer 
7cececd7: Pulling fs layer 
Digest: sha256:48318407b8d98e8c7d5bd4741c88e8e1a5442de660b47f63ba656e5c910bc3da
Status: Downloaded newer image for curlimages/curl:7.88.1
*   Trying 172.18.0.2:8080...
* Connected to restjpa-app-1 (172.18.0.2) port 8080 (#0)
> GET /restjpa/persons HTTP/1.1
> Host: restjpa-app-1:8080
> User-Agent: curl/7.88.1-DEV
> Accept: application/json
> 
< HTTP/1.1 200 OK
< Content-Type: application/json
< Content-Length: 187
< 
{ [187 bytes data]
* Connection #0 to host restjpa-app-1 left intact
[{"email":"a.b@ici.fr","firstname":"a","id":1,"lastname":"b"},{"email":"c.d@la.fr","firstname":"c","id":2,"lastname":"d"},{"email":"e.f@enc

### Un exemple avancé

L'exemple ci-dessous présente un exemple un peu plus avancé en ajoutant un reverse proxy (https://traefik.io) pour gérer les points d'entrées de l'application (sécurité, répartition de charges, ...).

```{literalinclude} sample-java/restjpa/Docker-compose-proxy.yml
:language: docker-compose
:caption: sample-java/restjpa/Docker-compose-proxy.yml
:name: sample-java/restjpa/Docker-compose-proxy.yml
```

La directive `healthcheck` de docker compose (qui existe dans le client) permet d'indique une commande à exécuter pour vérifier l'état de santé du conteneur (s'il fonctionne correctement et pas seulement s'il a démarré). Cela peut être utilisé comme condition pour `depends_on`pour ne pas simple lancer un autre service mais attendre qu'il soit fonctionnel.

In [ ]:
( cd sample-java/restjpa && \
  docker compose --file Docker-compose.yml up --quiet-pull --detach )